In [1]:
# Import libraries
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import RMSprop
from sklearn.model_selection import train_test_split

In [2]:
# Set random seed for deterministic result
torch.manual_seed(0)
np.random.seed(0)

In [3]:
import pandas as pd
df = pd.read_csv('diabetes.csv')
# Create training and test sets
# .values convert form pandas to numpy array
X = df.iloc[:,0:8].values
y = df.iloc[:,8].values

from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(X, y, test_size = 0.2)

In [4]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
features_train = sc.fit_transform(features_train)
features_test = sc.transform(features_test)

In [5]:
# Convert data to PyTorch tensors
x_train = torch.from_numpy(features_train).float()
# .view(-1, 1): This method reshapes tensor to have one column 
#and as many rows as needed to maintain the total number of elements. 
#The -1 argument automatically infers the correct number of rows.
y_train = torch.from_numpy(target_train).float().view(-1, 1)
x_test = torch.from_numpy(features_test).float()
y_test = torch.from_numpy(target_test).float().view(-1, 1)

In [6]:
# Define a neural network using `Sequential`
class SimpleNeuralNet(nn.Module):
    def __init__(self):
        super(SimpleNeuralNet, self).__init__()
        self.sequential = torch.nn.Sequential(
        torch.nn.Linear(8, 12),
        torch.nn.ReLU(),
        torch.nn.Linear(12,8),
        torch.nn.ReLU(),
        torch.nn.Linear(8, 1),
        torch.nn.Sigmoid()
        )
    def forward(self, x):
        x = self.sequential(x)
        return x

In [7]:
# Initialize neural network
network = SimpleNeuralNet()
# Define loss function, optimizer
criterion = nn.BCELoss()
#optimizer = RMSprop(network.parameters())
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)

In [8]:
# Define data loader
train_data = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_data, batch_size=10, shuffle=True)
# Compile the model using torch 2.0's optimizer
network = torch.compile(network)

In [9]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

# Train neural network
epochs = 150
for epoch in range(epochs):
#This loop iterates over batches of data from the train_loader. 
# data contains the x_train, and target the y_train.    
    for batch_idx, (data, target) in enumerate(train_loader):
# clears possible old gradients from previous last step        
        optimizer.zero_grad()
# network processes input data and produces an output        
        output = network(data)
# computes the loss         
        loss = criterion(output, target)
# computes the gradient of the loss with respect to the network parameters        
        loss.backward()
# optimizer updates the model's parameters based on the gradients calculated in the previous step        
        optimizer.step()
    print("Epoch:", epoch+1, "\tLoss:", loss.item())

W0618 10:39:53.064000 5124 torch\_dynamo\convert_frame.py:824] WON'T CONVERT forward C:\Users\tdiak_000\AppData\Local\Temp\ipykernel_11124\3220728854.py line 13 
W0618 10:39:53.064000 5124 torch\_dynamo\convert_frame.py:824] due to: 
W0618 10:39:53.064000 5124 torch\_dynamo\convert_frame.py:824] Traceback (most recent call last):
W0618 10:39:53.064000 5124 torch\_dynamo\convert_frame.py:824]   File "C:\Users\tdiak_000\anaconda3\Lib\site-packages\torch\_dynamo\convert_frame.py", line 786, in _convert_frame
W0618 10:39:53.064000 5124 torch\_dynamo\convert_frame.py:824]     result = inner_convert(
W0618 10:39:53.064000 5124 torch\_dynamo\convert_frame.py:824]              ^^^^^^^^^^^^^^
W0618 10:39:53.064000 5124 torch\_dynamo\convert_frame.py:824]   File "C:\Users\tdiak_000\anaconda3\Lib\site-packages\torch\_dynamo\convert_frame.py", line 400, in _convert_frame_assert
W0618 10:39:53.064000 5124 torch\_dynamo\convert_frame.py:824]     return _compile(
W0618 10:39:53.064000 5124 torch\_dyn

Epoch: 1 	Loss: 0.6616560816764832
Epoch: 2 	Loss: 0.6342306733131409
Epoch: 3 	Loss: 0.48842716217041016
Epoch: 4 	Loss: 0.334125280380249
Epoch: 5 	Loss: 0.7391358613967896
Epoch: 6 	Loss: 0.6147226691246033
Epoch: 7 	Loss: 0.26613762974739075
Epoch: 8 	Loss: 0.615837812423706
Epoch: 9 	Loss: 0.4969792664051056
Epoch: 10 	Loss: 0.22612985968589783
Epoch: 11 	Loss: 0.685439944267273
Epoch: 12 	Loss: 0.6774601340293884
Epoch: 13 	Loss: 0.3338824510574341
Epoch: 14 	Loss: 0.5040731430053711
Epoch: 15 	Loss: 0.31876030564308167
Epoch: 16 	Loss: 0.3050353229045868
Epoch: 17 	Loss: 0.6880777478218079
Epoch: 18 	Loss: 0.9291672706604004
Epoch: 19 	Loss: 0.3535291254520416
Epoch: 20 	Loss: 0.36819523572921753
Epoch: 21 	Loss: 0.38057273626327515
Epoch: 22 	Loss: 0.22364917397499084
Epoch: 23 	Loss: 0.49596458673477173
Epoch: 24 	Loss: 0.5005338788032532
Epoch: 25 	Loss: 0.4395853877067566
Epoch: 26 	Loss: 0.41832926869392395
Epoch: 27 	Loss: 0.179815411567688
Epoch: 28 	Loss: 0.3427367210388

In [10]:
# Evaluate neural network
with torch.no_grad():
    output = network(x_test)
    test_loss = criterion(output, y_test)
    test_accuracy = (output.round() == y_test).float().mean()


In [11]:
    print("Test Loss:", test_loss.item(), "\tTest Accuracy:",
        test_accuracy.item())

Test Loss: 0.4881787598133087 	Test Accuracy: 0.7792207598686218


In [12]:
test_loss.item()

0.4881787598133087

In [13]:
test_accuracy

tensor(0.7792)